In [ ]:
from pathlib import Path

import numpy as np
import torch
from util_plot import plot_fc

# import os
# os.environ["TIREX_NO_CUDA"] = "1"   # Experimental!!: Turns off sLSTM CUDA kernels if you have problems but be aware of the downsides! (see repository FAQ)
from tirex import ForecastModel, load_model

# Load Example Data
ctx_s, future_s = np.split(np.genfromtxt(Path.cwd() / "air_passengers.csv"), [-12])
ctx_l, future_l = np.split(np.genfromtxt(Path.cwd() / "loop_seattle_5T_example.csv"), [-512])

### TiRex Forecast in 2 Lines

1) Load Model
2) Generate Forecast

In [ ]:
# 1) Load Model
model: ForecastModel = load_model("NX-AI/TiRex")

# 2) Short Horizon - Example
quantiles, mean = model.forecast(ctx_s, prediction_length=24)
plot_fc(ctx_s, quantiles[0], future_s)

# 2) Long Horizon - Example
quantiles, mean = model.forecast(ctx_l, prediction_length=768)
plot_fc(ctx_l, quantiles[0], future_l)

### Input Options

TiRex supports forecasting with different input types

In [ ]:
data = torch.tensor(np.genfromtxt(Path.cwd() / "air_passengers.csv"))  # Load Example

# Torch tensor (2D or 1D)
quantiles, means = model.forecast(context=data, prediction_length=24)
print("Predictions (Torch tensor):\n", type(quantiles), quantiles.shape)

# List of Torch tensors (List of 1D) - will be padded
list_torch_data = [data, data, data]
quantiles, means = model.forecast(context=list_torch_data, prediction_length=24, batch_size=2)
print("Predictions (List of Torch tensors):\n", type(quantiles), quantiles.shape)

# NumPy array (2D or 1D)
quantiles, means = model.forecast(context=data.numpy(), prediction_length=24, output_type="torch")
print("Predictions (NumPy):\n", type(quantiles), quantiles.shape)


# List of NumPy arrays (List of 1D) - will be padded
list_numpy_data = [data.numpy()]  # Split into 3 sequences
quantiles, means = model.forecast(context=list_numpy_data, prediction_length=24)
print("Predictions (List of NumPy arrays):\n", type(quantiles), quantiles.shape)


# GluonTS Dataset
try:
    from typing import cast

    from gluonts.dataset import Dataset

    gluon_dataset = cast(Dataset, [{"target": data, "item_id": 1}, {"target": data, "item_id": 22}])
    quantiles, means = model.forecast_gluon(gluon_dataset, prediction_length=24)
    print("Predictions GluonDataset:\n", type(quantiles), quantiles.shape)
    # If you use also `glutonts` as your output type the start_time and item_id get preserved accordingly
    predictions_gluon = model.forecast_gluon(gluon_dataset, prediction_length=24, output_type="gluonts")
    print("Predictions GluonDataset:\n", type(predictions_gluon), type(predictions_gluon[0]))
except Exception as e:
    print(e)
    # To use the gluonts function you need to install the optional dependency
    # pip install tirex[gluonts]
    pass

### Output Options


TiRex supports different output types for the forecasts

In [ ]:
data = torch.tensor(np.genfromtxt(Path.cwd() / "air_passengers.csv"))  # Load Example

# Default: 2D Torch tensor
quantiles, means = model.forecast(context=data, prediction_length=24, output_type="torch")
print("Predictions:\n", type(quantiles), quantiles.shape)


# 2D Numpy Array
quantiles, means = model.forecast(context=data, prediction_length=24, output_type="numpy")
print("Predictions:\n", type(quantiles), quantiles.shape)


# Iterate by patch
# You can also use the forecast function as iterable. This might help with big datasets. All output_types are supported
for i, fc_batch in enumerate(
    model.forecast(context=[data, data, data, data, data], batch_size=2, output_type="torch", yield_per_batch=True)
):
    quantiles, means = fc_batch
    print(f"Predictions batch {i}:\n", type(quantiles), quantiles.shape)


try:
    # QuantileForecast (GluonTS)
    predictions_gluonts = model.forecast(context=data, prediction_length=24, output_type="gluonts")
    print("Predictions (GluonTS Quantile Forecast):\n", type(predictions_gluon), type(predictions_gluon[0]))
    predictions_gluonts[0].plot()
except Exception as e:
    print(e)
    # To use the gluonts function you need to install the optional dependency
    # pip install tirex[gluonts]